In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [7]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras.datasets import cifar10

In [8]:
(train_ds, validation_ds, test_ds), info = tfds.load('cifar10',
                                    split=["train", "test[:35%]", "test[35%:]"],
                                    as_supervised = True,
                                    with_info=True,
                                    shuffle_files= True)

In [9]:
size = (32, 32)
batch_size = 128

def normalize_resize(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.divide(image, 255)
    image = tf.image.resize(image, size)
    return image, label

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_saturation(image, 0.7, 1.3)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_brightness(image, 0.1)
    image = tf.image.random_crop(image, (32, 32, 3))
    return image, label 

train_ds = train_ds.map(normalize_resize).cache().map(augment).batch(batch_size).prefetch(buffer_size=1)
validation_ds = validation_ds.map(normalize_resize).cache().batch(batch_size).prefetch(buffer_size=1)
test_ds = test_ds.map(normalize_resize).cache().batch(batch_size).prefetch(buffer_size=1)

In [ ]:
%cd  /home/reynier/py_projects/Deployments/functions

In [10]:
from Nets_rey import GetNeuralNetworkModel
from Stats import WeightQuantization, ActivationStats, CheckAccuracyAndLoss, QuantizationEffect, GetReadAndWrites
from funciones import buffer_vectores
from Simulation import save_obj, load_obj

In [11]:
model = GetNeuralNetworkModel('VGG16',(32,32,3),10, quantization = False, aging_active=False)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
lambda (Lambda)              (None, 32, 32, 3)         0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
lambda_2 (Lambda)            (None, 32, 32, 64)        0         
_________________________________________________________________
re_lu (ReLU)                 (None, 32, 32, 64)        0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 32, 32, 64)        0     

In [13]:
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [14]:
model.compile(
    optimizer=Adam(0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [15]:
pwd

'C:\\Users\\usuario\\Desktop\\CNN_Gating'

In [10]:
cd Data\Trained_Weights_imagNet\VGG16\

C:\Users\usuario\Desktop\CNN_Gating\Data\Trained_Weights_imagNet\VGG16


In [ ]:
#%cd  /tmp/Yami_Weights/VGG16/

In [11]:
model.load_weights('weights.data')

In [12]:
score = model.evaluate(test_ds, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

NotFoundError: NewRandomAccessFile failed to Create/Open: ~\tensorflow_datasets\cifar10\3.0.2\cifar10-test.tfrecord-00000-of-00001 : El sistema no puede encontrar la ruta especificada.
; No such process [Op:IteratorGetNext]

In [ ]:
score = model.evaluate(train_ds, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])